In [ ]:
pip install yfinance scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 176.3 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [43]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.linalg import cholesky
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Step 1: Retrieve historical data
def get_historical_data(tickers, start_date, end_date):
    # Create a dictionary to store valid time-series data
    data = {}
    
    for ticker in tickers:
        print(f"Fetching data for {ticker}...")
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        
        # Ensure we fetch the "Close" column as a pandas Series
        if not stock_data.empty:
            data[ticker] = stock_data['Close']
            print(f"Data fetched for {ticker}: {len(stock_data)} rows.")
        else:
            print(f"Warning: No data fetched for {ticker}. Skipping.")
    
    # Check if any data was fetched
    if not data:
        raise ValueError("No valid data fetched for any tickers. Please check your tickers or date range.")
    
    # Combine all Series into a DataFrame, aligning indices
    try:
        df = pd.concat(data, axis=1)  # Combine the Series into a DataFrame
        df.columns = tickers  # Set the column names as tickers
        print("DataFrame successfully created.")
        
        # Drop rows with NaN values (if any tickers have missing data)
        if df.isnull().values.any():
            print("Warning: DataFrame contains missing values. Dropping rows with NaN values.")
            df.dropna(inplace=True)
        
        if df.empty:
            raise ValueError("DataFrame is empty after dropping NaN values. Check the input data.")
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        raise ValueError("Unable to create DataFrame from the fetched data.")
    
    print("Data fetching and cleaning complete.")
    return df



In [44]:
# Step 2: Calculate volatility and correlation
def calculate_stats(data):
    log_returns = np.log(data / data.shift(1)).dropna()
    volatility = log_returns.std() * np.sqrt(252)  # Annualized volatility
    correlation = log_returns.corr()
    return log_returns, volatility, correlation



In [45]:
# Step 3: Simulate price paths using GBM
def simulate_paths(S0, mu, sigma, corr_matrix, T, dt, num_simulations):
    n_steps = int(T / dt)
    n_assets = len(S0)
    L = cholesky(corr_matrix, lower=True)
    paths = np.zeros((num_simulations, n_steps + 1, n_assets))
    paths[:, 0, :] = S0

    for t in range(1, n_steps + 1):
        Z = np.random.normal(0, 1, (num_simulations, n_assets))
        correlated_Z = np.dot(Z, L.T)
        dS = (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * correlated_Z
        paths[:, t, :] = paths[:, t - 1, :] * np.exp(dS)

    return paths



In [46]:
# Step 4: Evaluate early redemption conditions
def evaluate_redemption(paths, thresholds, redemption_dates):
    num_simulations, n_steps, n_assets = paths.shape
    redemption_payoff = np.zeros(num_simulations)
    for i in range(num_simulations):
        for j, threshold in enumerate(thresholds):
            if all(paths[i, redemption_dates[j], :] >= threshold):
                redemption_payoff[i] = 1.044  # Early redemption with coupon
                break
        if redemption_payoff[i] == 0:
            # Calculate final payoff at maturity
            final_prices = paths[i, -1, :]
            min_final_ratio = final_prices / thresholds[0]
            if min_final_ratio.min() < 0.625:
                redemption_payoff[i] = min_final_ratio.min()  # Principal loss
            else:
                redemption_payoff[i] = 1.132  # Full principal with coupon
    return redemption_payoff



In [47]:
# Step 5: Discount payoff to present value
def discount_payoffs(payoffs, risk_free_rate, T):
    discounted_payoffs = payoffs * np.exp(-risk_free_rate * T)
    return discounted_payoffs


In [48]:
# Function to calculate downside deviation
def calculate_downside_deviation(returns, target=0):
    downside_diff = np.minimum(0, returns - target)  # Only consider negative deviations
    downside_deviation = np.sqrt(np.mean(downside_diff**2))  # Square and average
    return downside_deviation



In [49]:
# Evaluate early redemption conditions and calculate returns and downside deviations
def evaluate_with_early_redemption(simulated_paths, thresholds, redemption_dates):
    num_simulations, n_steps, n_assets = simulated_paths.shape
    redemption_payoff = np.zeros(num_simulations)
    early_redemption_times = np.full(num_simulations, T)  # Default to maturity (T years)
    all_returns = []

    for i in range(num_simulations):
        for j, threshold in enumerate(thresholds):
            # Check early redemption only on specified dates
            if redemption_dates[j] < n_steps and all(simulated_paths[i, redemption_dates[j], :] >= threshold):
                redemption_payoff[i] = 1.044  # Early redemption with coupon
                early_redemption_times[i] = redemption_dates[j] / 252  # Convert days to years
                break

        # If no early redemption occurred, calculate final payoff at maturity
        if redemption_payoff[i] == 0:
            final_prices = simulated_paths[i, -1, :]
            min_final_ratio = final_prices / thresholds[0]
            if min_final_ratio.min() < 0.625:
                redemption_payoff[i] = min_final_ratio.min()  # Principal loss
            else:
                redemption_payoff[i] = 1.132  # Full principal with coupon
        
        # Calculate returns for each asset over the entire path
        returns = (simulated_paths[i, :, :] / simulated_paths[i, 0, :]) - 1
        all_returns.append(returns)

    all_returns = np.array(all_returns)
    return redemption_payoff, early_redemption_times, all_returns


In [50]:
# Function to run the entire simulation process for a given period
def run_simulation(tickers, start_date, end_date, S0, thresholds, redemption_dates, T, dt, num_simulations, risk_free_rate):
    print(f"\nRunning Simulation for {start_date} to {end_date}")
    
    # Fetch historical data
    historical_data = get_historical_data(tickers, start_date, end_date)
    log_returns, volatilities, corr_matrix = calculate_stats(historical_data)

    # Simulate price paths
    mu = log_returns.mean().values  # Average log returns
    sigma = volatilities.values     # Volatility
    simulated_paths = simulate_paths(S0, mu, sigma, corr_matrix.values, T, dt, num_simulations)

    # Evaluate redemption and calculate payoffs
    payoffs, early_redemptions, all_returns = evaluate_with_early_redemption(
        simulated_paths, thresholds, redemption_dates
    )

    # Calculate expected returns and downside deviations
    expected_returns = all_returns.mean(axis=0).mean(axis=0)  # Average return for each asset
    downside_deviations = [calculate_downside_deviation(all_returns[:, :, i].flatten()) for i in range(len(S0))]

    # Discount payoffs
    discounted_payoffs = discount_payoffs(payoffs, risk_free_rate, T)
    ELS_value = discounted_payoffs.mean()

    # Return results
    return {
        "ELS_Value": ELS_value,
        "Expected_Returns": expected_returns,
        "Downside_Deviations": downside_deviations,
        "Early_Redemptions_Avg_Time": np.mean(early_redemptions),
    }
import matplotlib.pyplot as plt



In [51]:
# Function to visualize results
def visualize_results(pre_results, post_results, issuance_results, asset_names):
    # Plot ELS Values
    plt.figure(figsize=(10, 6))
    periods = ['Pre-COVID', 'Post-COVID', 'Issuance Date']
    values = [pre_results['ELS_Value'], post_results['ELS_Value'], issuance_results['ELS_Value']]
    plt.bar(periods, values, color=['blue', 'orange', 'green'], alpha=0.7)
    plt.title("ELS Values Across Periods")
    plt.ylabel("ELS Value")
    plt.grid()
    plt.show()

    # Plot Expected Returns
    for i, asset in enumerate(asset_names):
        plt.figure(figsize=(10, 6))
        returns = [
            pre_results['Expected_Returns'][i],
            post_results['Expected_Returns'][i],
            issuance_results['Expected_Returns'][i],
        ]
        plt.bar(periods, returns, color=['blue', 'orange', 'green'], alpha=0.7)
        plt.title(f"Expected Returns for {asset}")
        plt.ylabel("Expected Return")
        plt.grid()
        plt.show()

    # Plot Downside Deviations
    for i, asset in enumerate(asset_names):
        plt.figure(figsize=(10, 6))
        risks = [
            pre_results['Downside_Deviations'][i],
            post_results['Downside_Deviations'][i],
            issuance_results['Downside_Deviations'][i],
        ]
        plt.bar(periods, risks, color=['blue', 'orange', 'green'], alpha=0.7)
        plt.title(f"Downside Deviations for {asset}")
        plt.ylabel("Downside Deviation")
        plt.grid()
        plt.show()





In [52]:
# Function to save results and visualize Monte Carlo simulations
def save_and_visualize_monte_carlo(pre_results, post_results, issuance_results, simulated_paths, asset_names, output_file="simulation_results.xlsx"):
    # Prepare results table
    periods = ["Pre-COVID", "Post-COVID", "Issuance Date"]
    results_table = pd.DataFrame({
        "Period": periods,
        "ELS Value": [pre_results['ELS_Value'], post_results['ELS_Value'], issuance_results['ELS_Value']],
        "Expected Returns KOSPI200": [pre_results['Expected_Returns'][0], post_results['Expected_Returns'][0], issuance_results['Expected_Returns'][0]],
        "Expected Returns HSCEI": [pre_results['Expected_Returns'][1], post_results['Expected_Returns'][1], issuance_results['Expected_Returns'][1]],
        "Expected Returns S&P500": [pre_results['Expected_Returns'][2], post_results['Expected_Returns'][2], issuance_results['Expected_Returns'][2]],
        "Downside Deviation KOSPI200": [pre_results['Downside_Deviations'][0], post_results['Downside_Deviations'][0], issuance_results['Downside_Deviations'][0]],
        "Downside Deviation HSCEI": [pre_results['Downside_Deviations'][1], post_results['Downside_Deviations'][1], issuance_results['Downside_Deviations'][1]],
        "Downside Deviation S&P500": [pre_results['Downside_Deviations'][2], post_results['Downside_Deviations'][2], issuance_results['Downside_Deviations'][2]],
        "Avg Redemption Time (Years)": [pre_results['Early_Redemptions_Avg_Time'], post_results['Early_Redemptions_Avg_Time'], issuance_results['Early_Redemptions_Avg_Time']]
    })

    # Save to Excel
    results_table.to_excel(output_file, index=False)
    print(f"Results saved to {output_file}")

    # Visualize Monte Carlo paths for the first asset (KOSPI200) as an example
    plt.figure(figsize=(12, 6))
    plt.plot(simulated_paths[:, :, 0].T, alpha=0.1, color='blue')  # Plot paths for KOSPI200
    plt.title("Monte Carlo Simulation Paths for KOSPI200")
    plt.xlabel("Time Steps")
    plt.ylabel("Price")
    plt.grid(True)
    plt.show()

    # Visualize results table as a heatmap
    plt.figure(figsize=(12, 6))
    sns.heatmap(results_table.iloc[:, 1:].set_index(results_table["Period"]), annot=True, cmap="coolwarm", fmt=".3f")
    plt.title("Comparison of Key Metrics Across Periods")
    plt.show()


In [53]:
# Adjust formatting for the results table
def format_results_table(pre_results, post_results, issuance_results, asset_names):
    periods = ["Pre-COVID", "Post-COVID", "Issuance Date"]
    formatted_table = pd.DataFrame({
        "Period": periods,
        "ELS Value": [
            round(pre_results['ELS_Value'], 4),
            round(post_results['ELS_Value'], 4),
            round(issuance_results['ELS_Value'], 4)
        ],
        **{
            f"Expected Returns ({asset})": [
                round(pre_results['Expected_Returns'][i], 4),
                round(post_results['Expected_Returns'][i], 4),
                round(issuance_results['Expected_Returns'][i], 4)
            ] for i, asset in enumerate(asset_names)
        },
        **{
            f"Downside Deviation ({asset})": [
                round(pre_results['Downside_Deviations'][i], 4),
                round(post_results['Downside_Deviations'][i], 4),
                round(issuance_results['Downside_Deviations'][i], 4)
            ] for i, asset in enumerate(asset_names)
        },
        "Avg Redemption Time (Years)": [
            round(pre_results['Early_Redemptions_Avg_Time'], 4),
            round(post_results['Early_Redemptions_Avg_Time'], 4),
            round(issuance_results['Early_Redemptions_Avg_Time'], 4)
        ]
    })
    return formatted_table





In [54]:

# Monte Carlo visualization for all assets
def visualize_monte_carlo_all_assets(simulated_paths, asset_names):
    for i, asset_name in enumerate(asset_names):
        plt.figure(figsize=(12, 6))
        plt.plot(simulated_paths[:, :, i].T, alpha=0.1, label=asset_name, color=f"C{i}")  # Plot paths for each asset
        plt.title(f"Monte Carlo Simulation Paths for {asset_name}")
        plt.xlabel("Time Steps")
        plt.ylabel("Price")
        plt.grid(True)
        plt.legend()
        plt.show()

In [56]:

# Main Program
if __name__ == "__main__":
    # Define simulation parameters
    tickers = ['^KS11', '^HSI', '^GSPC']  # KOSPI200, HSCEI, S&P500
    asset_names = ['KOSPI200', 'HSCEI', 'S&P500']
    S0 = np.array([275.82, 10533.24, 3037.56])  # Initial prices for KOSPI200, HSCEI, S&P500
    redemption_thresholds = [0.9, 0.85, 0.8, 0.75, 0.625]  # Redemption thresholds as percentages
    thresholds = [S0 * t for t in redemption_thresholds]
    redemption_dates = [126, 252, 378, 504, 630]  # Approx. 6-month intervals in trading days
    T = 3  # Maturity in years
    dt = 1 / 252  # Daily time step
    num_simulations = 10000
    risk_free_rate = 0.02

    # Define periods
    pre_covid_period = ("2015-01-01", "2017-12-31")  # Pre-COVID-19
    post_covid_period = ("2020-01-01", "2022-12-31")  # Post-COVID-19
    issuance_date_period = ("2019-10-25", "2019-11-05")  # Expanded Issuance Date Range

    # Run simulation for pre-COVID-19
    pre_covid_results = run_simulation(
        tickers, pre_covid_period[0], pre_covid_period[1], S0, thresholds, redemption_dates, T, dt, num_simulations, risk_free_rate
    )

    # Run simulation for post-COVID-19
    post_covid_results = run_simulation(
        tickers, post_covid_period[0], post_covid_period[1], S0, thresholds, redemption_dates, T, dt, num_simulations, risk_free_rate
    )

    # Run simulation for issuance date
    issuance_results = run_simulation(
        tickers, issuance_date_period[0], issuance_date_period[1], S0, thresholds, redemption_dates, T, dt, num_simulations, risk_free_rate
    )

    # Format results table
    formatted_results_table = format_results_table(pre_covid_results, post_covid_results, issuance_results, asset_names)

    # Save the results table to an Excel file
    formatted_results_table.to_excel("Formatted_ELS_Simulation_Results.xlsx", index=False)
    print("Results saved to 'Formatted_ELS_Simulation_Results.xlsx'")

    # Display the formatted table to the user
    print(formatted_results_table)

    # Visualize Monte Carlo simulations for all assets
    #simulated_paths_placeholder = np.random.rand(1000, 252, 3) * 100  # Example paths for visualization
    #visualize_monte_carlo_all_assets(simulated_paths_placeholder, asset_names)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Running Simulation for 2015-01-01 to 2017-12-31
Fetching data for ^KS11...
Data fetched for ^KS11: 735 rows.
Fetching data for ^HSI...
Data fetched for ^HSI: 738 rows.
Fetching data for ^GSPC...
Data fetched for ^GSPC: 755 rows.
DataFrame successfully created.
Data fetching and cleaning complete.



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Running Simulation for 2020-01-01 to 2022-12-31
Fetching data for ^KS11...
Data fetched for ^KS11: 740 rows.
Fetching data for ^HSI...
Data fetched for ^HSI: 741 rows.
Fetching data for ^GSPC...
Data fetched for ^GSPC: 756 rows.
DataFrame successfully created.
Data fetching and cleaning complete.



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Running Simulation for 2019-10-25 to 2019-11-05
Fetching data for ^KS11...
Data fetched for ^KS11: 7 rows.
Fetching data for ^HSI...
Data fetched for ^HSI: 7 rows.
Fetching data for ^GSPC...
Data fetched for ^GSPC: 7 rows.
DataFrame successfully created.
Data fetching and cleaning complete.


Results saved to 'Formatted_ELS_Simulation_Results.xlsx'
          Period  ELS Value  Expected Returns (KOSPI200)  \
0      Pre-COVID     0.9741                       0.0025   
1     Post-COVID     0.8972                      -0.0038   
2  Issuance Date     0.9825                       0.0070   

   Expected Returns (HSCEI)  Expected Returns (S&P500)  \
0                    0.0018                     0.0021   
1                   -0.0028                     0.0039   
2                    0.0091                     0.0038   

   Downside Deviation (KOSPI200)  Downside Deviation (HSCEI)  \
0                         0.0953                      0.1410   
1                         0.1752                      0.2038   
2                         0.0876                      0.0997   

   Downside Deviation (S&P500)  Avg Redemption Time (Years)  
0                       0.1021                       0.5108  
1                       0.1980                       1.1483  
2                       0.

# 시뮬레이션 결과 분석 및 시사점

## 1. ELS Value (ELS 가치)
- **Pre-COVID**: **0.9741**
  - 팬데믹 이전 시장은 비교적 안정적이었으며, ELS의 평균 가치는 원금(1.0)에 근접.
- **Post-COVID**: **0.8972**
  - 팬데믹 이후 높은 변동성으로 인해 조기 상환이 어려워지고, 만기 손실 발생 시나리오 증가.
  - 원금 대비 약 **10.3% 감소**.
- **Issuance Date (발행 시점 기준)**: **0.9825**
  - 발행 시점에는 팬데믹 여파가 일부 완화되었으며, 시장이 안정적으로 반등.

### 시사점
- **ELS 가치는 시장 안정성과 밀접하게 연관**되어 있음.
- **발행 시점의 시장 상황이 중요**하며, 외부 충격(예: 팬데믹)은 큰 리스크로 작용.

---

## 2. Expected Returns (기대 수익률)
### KOSPI200
- **Pre-COVID**: **0.0025** (연환산 약 0.25%)
- **Post-COVID**: **-0.0038** (연환산 약 -0.38%)
- **Issuance Date**: **0.0070** (연환산 약 0.7%)

### HSCEI
- **Pre-COVID**: **0.0018** (연환산 약 0.18%)
- **Post-COVID**: **-0.0028** (연환산 약 -0.28%)
- **Issuance Date**: **0.0091** (연환산 약 0.91%)

### S&P500
- **Pre-COVID**: **0.0021** (연환산 약 0.21%)
- **Post-COVID**: **0.0039** (연환산 약 0.39%)
- **Issuance Date**: **0.0038** (연환산 약 0.38%)

### 시사점
- **자산별 성과 차이**가 명확히 드러남:
  - S&P500은 상대적으로 안정적인 수익률 유지.
  - KOSPI200과 HSCEI는 하락세로 인해 기대 수익률이 부정적.
- **하락장에서는 투자 손실 가능성**이 매우 큼.

---

## 3. Downside Deviation (하방 편차)
### KOSPI200
- **Pre-COVID**: **0.0953**
- **Post-COVID**: **0.1752**
- **Issuance Date**: **0.0876**

### HSCEI
- **Pre-COVID**: **0.1410**
- **Post-COVID**: **0.2038**
- **Issuance Date**: **0.0997**

### S&P500
- **Pre-COVID**: **0.1021**
- **Post-COVID**: **0.1980**
- **Issuance Date**: **0.0582**

### 시사점
- 팬데믹 이후 **HSCEI와 KOSPI200의 하방 리스크가 크게 증가**.
- **S&P500**은 상대적으로 안정적이나, 팬데믹 이후 변동성 증가.
- **Post-COVID**와 같은 시장 변동성이 큰 시기에는 ELS 투자 시 원금 손실 가능성이 높아짐.

---

## 4. Avg Redemption Time (평균 조기 상환 시간)
- **Pre-COVID**: **0.5108년** (약 6개월)
- **Post-COVID**: **1.1483년** (약 14개월)
- **Issuance Date**: **0.2209년** (약 2~3개월)

### 시사점
- **조기 상환 시간의 증가는 리스크 노출 기간 증가**를 의미.
- 시장이 안정적일수록 조기 상환이 빠르게 이루어짐.

---

# 종합적인 시사점
1. **시장 상황의 중요성**:
   - ELS 성과는 **시장 변동성**과 **자산의 성과**에 크게 의존.
   - 변동성이 큰 시장(Post-COVID)에서는 원금 손실 가능성이 높음.

2. **글로벌 vs. 지역 리스크**:
   - **S&P500**은 상대적으로 안정적이나, **HSCEI와 KOSPI200**은 지역적 리스크에 민감.

3. **발행 시점의 중요성**:
   - 시장 안정기나 상승세에 발행된 ELS가 높은 성과를 기대할 수 있음.

4. **ELS 구조적 특성**:
   - 조기 상환 조건이 기대 수익률을 제한하지만, 시장 하락 시 손실을 줄이는 데 기여.

5. **포트폴리오 다각화 필요성**:
   - 다양한 자산에 분산 투자하여 변동성을 줄이고 안정성을 높이는 것이 중요.




### Simulation Results
- **Pre-COVID (Stable Market)**:
  - Expected returns are modest, and downside risk is relatively low (e.g., KOSPI200 Expected Return: 0.0025, Downside Deviation: 0.0953).
- **Post-COVID (Volatile Market)**:
  - Expected returns turn negative (KOSPI200: -0.0038).
  - Downside risk more than doubles compared to Pre-COVID levels (KOSPI200: 0.1752).
- **Issuance Date**:
  - Expected returns recover (KOSPI200: 0.0070).
  - Downside risk decreases (KOSPI200: 0.0876).

### Implications
- **ELS performance is highly dependent on market conditions.**
- In bearish markets, the risk of losses is significantly higher than the potential for returns.
- ELS issued in stable markets generally performs better, with lower risks and higher chances of early redemption.

---

## 2. Impact of Early Redemption
### Research Findings
- As the **early redemption date approaches**, ELS value tends to **exceed its face value**, indicating higher likelihood of redemption.
- If early redemption fails, the value declines below the face value as maturity nears.
- Early redemption likelihood is a critical determinant of ELS value.

### Simulation Results
- **Average Early Redemption Time**:
  - Pre-COVID: **0.5108 years (about 6 months)**.
  - Post-COVID: **1.1483 years (about 14 months)**.
  - Issuance Date: **0.2209 years (about 2-3 months)**.
- Stable markets result in **higher early redemption probabilities**, while volatile markets delay redemption and increase risk.

### Implications
- **Early redemption heavily depends on market conditions**, with stable markets enabling faster redemption.
- Early redemption probabilities provide a key metric for evaluating ELS value.

---

## 3. Importance of Risk Management
### Research Findings
- As ELS has a fixed maturity, it is difficult to take flexible actions like cutting losses during a market downturn.
- ELS carries **higher risks than direct investments or indirect products like mutual funds**.
- Investors must sufficiently account for the risk of losses in adverse market conditions.

### Simulation Results
- During Post-COVID periods, downside deviation and volatility significantly increase, raising the likelihood of losses at maturity.
- Pre-COVID and issuance periods show reduced downside risks and higher probabilities of early redemption due to stable market conditions.

### Implications
- **Risk during volatile periods may be underestimated in ELS investments.**
- ELS products are better suited for stable or growth-oriented market environments.
- A robust risk management strategy is essential for addressing losses during market downturns.

---

## Comprehensive Conclusions
### Alignment Between Simulation Results and Research Findings
1. **Market Conditions Matter**:
   - ELS performance is heavily influenced by market volatility and asset performance.
   - In volatile markets (Post-COVID), risks escalate while early redemption becomes less likely.
2. **Global vs. Regional Risks**:
   - S&P500 demonstrates relative stability, whereas HSCEI and KOSPI200 are highly sensitive to regional market risks.
3. **Issuance Timing**:
   - ELS issued during stable markets offers better returns and higher probabilities of early redemption.
4. **Investor Considerations**:
   - ELS has inherent risks due to fixed maturity, making loss recovery challenging during downturns.
   - Investors need to assess potential losses carefully and prepare strategies for volatile markets.

### Recommendations
- **Portfolio Diversification**:
  - Diversify across multiple assets to reduce volatility and enhance stability.
- **Consider Market Conditions**:
  - ELS performs better when issued during stable or growth-driven markets.
- **Strengthen Risk Management**:
  - Evaluate early redemption probabilities and develop strategies for mitigating risks during bearish markets.
